In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from gensim.models.ldamodel import LdaModel
from gensim import corpora
import numpy as np
import math
import operator
import re
import datetime
# % matplotlib inline
# % matplotlib auto
path_dir = 'C:\\Users\\zhqch\\Documents\\code\\Python3Projects\\visual_detection\\lda\\'

C:\Software\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
conflict_df = pd.read_csv('C:\\Users\\zhqch\\Documents\\code\\Python3Projects\\visual_detection\\lda\\all_conflicts.csv')
# os.getcwd()
# conflict_df.head()
conflict_df = conflict_df.loc[conflict_df['PET'] > 0,:].loc[conflict_df['PET'] < 10,:]
conflict_df['PET_level'] = pd.cut(conflict_df['PET'],[0,1,2,3,4,5,6,7,8,9,10], labels=[1,2,3,4,5,6,7,8,9,10])
# plt.plot(conflict_df['PET'])
conflict_df['Word'] = conflict_df.apply(lambda x: str(x['Conflict_user'])+'_'+str(x['Vel_level']) +'_'+ str(x['Angle_level']) + '_'+str(x['PET_level']), axis =1)
conflict_df['Conflict_code'] = conflict_df.apply(lambda x: str(x['Conflict_user']) + '_' + str(x['Ped_id']) + '_' + str(x['Conflict_id']), axis=1)
doc_list = conflict_df['Conflict_code'].unique().tolist()
documents = []
for doc in doc_list:
    documents.append(conflict_df.loc[conflict_df['Conflict_code']==doc,'Word'].tolist())
# corpus
conflict_df['Word'].unique()

array(['2_2_2_10', '2_2_2_9', '2_2_2_8', '2_2_2_7', '2_2_1_7', '2_2_1_8',
       '2_2_1_10', '2_2_1_9', '2_2_1_6', '2_2_1_5', '2_2_1_4', '2_2_1_3',
       '2_2_1_2', '2_2_2_5', '2_2_2_6', '2_2_2_4', '2_3_1_10', '2_3_1_9',
       '2_3_1_8', '2_3_1_7', '2_3_1_6', '2_3_2_7', '1_2_1_10', '1_3_1_10',
       '1_3_1_9', '1_3_1_8', '1_3_1_7', '1_3_1_6', '1_3_1_5', '1_3_1_4',
       '1_3_1_3', '1_3_1_2', '1_3_1_1', '1_3_2_3', '1_2_2_9', '1_2_2_8',
       '1_2_2_10', '2_2_2_3', '2_2_2_2', '2_2_2_1', '1_2_2_7', '1_2_2_6',
       '1_2_2_5', '1_2_2_4', '1_2_2_3', '1_2_2_2', '1_2_1_9', '1_1_2_6',
       '1_1_1_6', '1_3_2_8', '1_3_2_9', '1_2_1_8', '1_2_1_7', '1_2_1_6',
       '1_2_1_5', '1_2_1_4', '1_3_2_4', '1_3_2_5', '1_3_2_6', '1_3_2_7',
       '1_3_2_10', '1_1_2_8', '1_1_2_9', '1_1_2_10', '2_2_1_1', '1_2_1_3',
       '1_2_2_1', '2_3_2_8', '2_3_1_5', '2_3_1_4', '1_2_1_2', '2_1_1_10',
       '2_1_1_9', '2_1_1_8', '2_1_1_7', '1_2_1_1', '1_1_2_7', '1_1_1_3',
       '1_1_1_4', '1_1_2_4', '2_3_2_10', '

In [59]:
conflict_df = conflict_df.loc[conflict_df['PET'] > 0,:].loc[conflict_df['PET'] < 10,:]
conflict_df['Word'].unique().shape


(120,)

In [3]:
# 创建语料的词语词典，每个单独的词语都会被赋予一个索引
dictionary = corpora.Dictionary(documents)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [5]:


def perplexity(ldamodel, testset, dictionary, size_dictionary, num_topics):
    """calculate the perplexity of a lda-model"""
    # dictionary : {7822:'deferment', 1841:'circuitry',19202:'fabianism'...]
    print ('the info of this ldamodel: \n')
    print ('num of testset: %s; size_dictionary: %s; num of topics: %s'%(len(testset), size_dictionary, num_topics))
    prep = 0.0
    prob_doc_sum = 0.0
    topic_word_list = [] # store the probablity of topic-word:[(u'business', 0.010020942661849608),(u'family', 0.0088027946271537413)...]
    for topic_id in range(num_topics):
        topic_word = ldamodel.show_topic(topic_id, size_dictionary)
        dic = {}
        for word, probability in topic_word:
            dic[word] = probability
        topic_word_list.append(dic)
    doc_topics_ist = [] #store the doc-topic tuples:[(0, 0.0006211180124223594),(1, 0.0006211180124223594),...]
    for doc in testset:
        doc_topics_ist.append(ldamodel.get_document_topics(doc, minimum_probability=0))
    testset_word_num = 0
    for i in range(len(testset)):
        prob_doc = 0.0 # the probablity of the doc
        doc = testset[i]
        doc_word_num = 0 # the num of words in the doc
        for word_id, num in doc:
            prob_word = 0.0 # the probablity of the word 
            doc_word_num += num
            word = dictionary[word_id]
            for topic_id in range(num_topics):
                # cal p(w) : p(w) = sumz(p(z)*p(w|z))
                prob_topic = doc_topics_ist[i][topic_id][1]
                prob_topic_word = topic_word_list[topic_id][word]
                prob_word += prob_topic*prob_topic_word
            prob_doc += math.log(prob_word) # p(d) = sum(log(p(w)))
        prob_doc_sum += prob_doc
        testset_word_num += doc_word_num
    prep = math.exp(-prob_doc_sum/testset_word_num) # perplexity = exp(-sum(p(d)/sum(Nd))
    print ("the perplexity of this ldamodel is : %s"%prep)
    return prep

# ppl = []
# for i in range(1, 30):
#     topic_num = i
#     doc_num = len(doc_list)
#     lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=topic_num, iterations=10000,  alpha=0.01, eta=0.01, minimum_probability=0.001,
#                             update_every = 1, chunksize = 100, passes = 1)
#     print('共训练%d篇文档，%d个主题' % (doc_num, topic_num))
#     ppl.append(perplexity(lda, corpus[:20], dictionary, len(dictionary), topic_num))
# pd.DataFrame(ppl).to_csv(path_dir+'perplexity_result.csv')
# plt.plot(range(1,5), ppl)
# plt.savefig('C:\\Users\\zhqch\\Documents\\code\\Python3Projects\\visual_detection\\lda\\perplexity.jpg')
# documents

C:\Software\Anaconda\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


共训练4782篇文档，1个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 1
the perplexity of this ldamodel is : 2.403523882692114


共训练4782篇文档，2个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 2
the perplexity of this ldamodel is : 2.1320398086665233


共训练4782篇文档，3个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 3
the perplexity of this ldamodel is : 1.9454412215872117


共训练4782篇文档，4个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 4
the perplexity of this ldamodel is : 1.936705024006207


...

共训练4782篇文档，25个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 25
the perplexity of this ldamodel is : 1.6511768549176589


共训练4782篇文档，26个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 26
the perplexity of this ldamodel is : 1.6574141502736954


共训练4782篇文档，27个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 27
the perplexity of this ldamodel is : 1.6702085299169709


共训练4782篇文档，28个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 28
the perplexity of this ldamodel is : 1.658576387993926


共训练4782篇文档，29个主题
the info of this ldamodel: 

num of testset: 20; size_dictionary: 120; num of topics: 29
the perplexity of this ldamodel is : 1.659640953550668


In [30]:
topic_num=12
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=topic_num, iterations=10000,  alpha=0.01, eta=0.01, minimum_probability=0.001,
                            update_every = 1, chunksize = 100, passes = 1)
# 主题分布
# print('***************************所有文档主题分布***************************')
doc_topics = lda.get_document_topics(corpus)
doc2topic_file = 'visual_detection//lda//doc2topic.csv'
with open(doc2topic_file, 'w') as f:
    f.write('***************************Topic distribution for each document***************************\n')

topic_list = []
# 输出每个文档的主题分布
for doc_num, topic in enumerate(doc_topics):
    one_doc_topic = [0 for _ in range(topic_num)]
    for topic_no, topic_probability in topic:
        one_doc_topic[topic_no] = topic_probability
    topic_list.append(one_doc_topic)
topic_df = pd.DataFrame(topic_list, index=doc_list)
topic_df.columns = ['Topic %d' % num for num in topic_df.columns]

cluster_model = KMeans(n_clusters = 4, n_jobs = 1, max_iter = 1000)
cluster_model.fit(topic_list)
topic_df['cluster'] = cluster_model.labels_
topic_df.to_csv(doc2topic_file, index_label='Conflict')
# with open(doc2topic_file, 'a') as f:
#             f.write('%s\t%d\t%.4f\n' % (doc_list[doc_num], topic_num, topic_probability))
# for topic_idx in topic_nums:
#     docs_in_topic = []  # 拥有该主题的所有doc
#     for doc_idx in range(doc_num):
#         for topic in doc_topics[doc_idx]:
#             if topic[0] == topic_idx:
#                 docs_in_topic.append([doc_idx, topic[1]])   # [文档id， 主题在该文档的占比]
#     docs_sorted = sorted(docs_in_topic, key = operator.itemgetter(1), reverse = True)
#     with open('visual_detection/lda/%s.txt' % time_now , 'a') as f:
#         temp_writing = '第%d个主题的排名前20个文档：\n' % topic_idx
#         for d in docs_sorted[:21]:
#             topic_in_d = doc_topics[d[0]]
#             topic_sorted = sorted(topic_in_d, key = operator.itemgetter(1), reverse = True)
#             temp_writing += '第%d个文档的主题分布：' % (d[0], ) + '\n'
#             for item in topic_sorted:
#                 temp_writing += '{:>8d}'.format(item[0])
#             temp_writing += '\n'
#             for item in topic_sorted[:10]:
#                temp_writing += '{:>8.4f}'.format(item[1])
#             temp_writing += '\n'
#         f.write(temp_writing)
    # print('第%d个文档的前%d个主题：' % (doc_id, 10), [item[0] for item in topic_sorted[:10]])
    # print( [item[1] for item in topic_sorted[:10]])

# 单词分布
topic2word = []
for topic_id in range(topic_num):
    word_prob = []
    term_distribute_all = lda.get_topic_terms(topicid=topic_id)
    term_distribute = term_distribute_all
    term_distribute = np.array(term_distribute)
    term_id = term_distribute[:, 0].astype(np.int)
    for ti,td in zip(term_id, term_distribute):
        word_prob.append([dictionary.id2token[ti], round(td[1],3)])
    topic2word += list(zip(*word_prob))
topic_df = pd.DataFrame(topic2word)
topic_df.to_csv('visual_detection/lda/topic2word.csv', index=False)
    
        
        
    
# print('***************************所有主题的单词分布***************************')
# with open('visual_detection/lda/topic2word.txt', 'w') as f:
#     f.write('***************************所有主题的单词分布***************************\n')

# for topic_id in range(topic_num):
#     term_distribute_all = lda.get_topic_terms(topicid=topic_id)
#     term_distribute = term_distribute_all
#     term_distribute = np.array(term_distribute)
#     term_id = term_distribute[:, 0].astype(np.int)
#     with open('visual_detection/lda/topic2word.txt', 'a') as f:
#         temp_writing = ''
#         temp_writing += '\n主题#%d：\t' % topic_id + '\n'
#         temp_writing += '词：\t'
#         for t in term_id:
#             temp_writing += dictionary.id2token[t] + '\t'
#         temp_writing += '\n概率：\t'
#         for td in term_distribute:
#             temp_writing += '{:>8.4f}'.format(td[1])
#         f.write(temp_writing)
    # print('主题#%d：\t' % topic_id)
    # print('词：\t',end = ' ')
    # for t in term_id:
        # print(dictionary.id2token[t],end = ' ')
    # print('\n概率：\t', term_distribute[:, 1])

C:\Software\Anaconda\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [29]:
a= [[1,2],[1,2]]
list(zip(*a))

[(1, 1), (2, 2)]

In [39]:
df1 = pd.read_csv('visual_detection//lda//doc2topic.csv')
df2 = pd.read_csv('visual_detection//lda//filter_conflicts.csv')
print(df1.head())
print(df2.head())

      Conflict   Topic 0  Topic 1  Topic 2  Topic 3  Topic 4  Topic 5  \
0  2_9732_6891  0.000000      0.0      0.0      0.0      0.0      0.0   
1  2_9732_6932  0.000000      0.0      0.0      0.0      0.0      0.0   
2  2_9757_6861  0.718030      0.0      0.0      0.0      0.0      0.0   
3  2_9757_6891  0.545257      0.0      0.0      0.0      0.0      0.0   
4  2_9757_6932  0.149792      0.0      0.0      0.0      0.0      0.0   

   Topic 6   Topic 7  Topic 8  Topic 9  Topic 10  Topic 11  cluster  
0      0.0  0.000000      0.0      0.0  0.992725       0.0        2  
1      0.0  0.239373      0.0      0.0  0.756646       0.0        2  
2      0.0  0.000000      0.0      0.0  0.276452       0.0        1  
3      0.0  0.118932      0.0      0.0  0.332080       0.0        1  
4      0.0  0.457081      0.0      0.0  0.389235       0.0        1  
   Ped_id  Time       Ped_x     Ped_y   Ped_vel  Vel_level  Conflict_user  \
0    9732  3189  194.668803  35.95238  1.245479          2      

In [42]:
print(df2.groupby('Conflict_code').mean().head())
# df = pd.merge(df1,df2,how='inner',left_on='Conflict',right_on='Conflict_id')
# print(df1.head())

               Ped_id          Time       Ped_x      Ped_y   Ped_vel  \
Conflict_code                                                          
1_1018_1061    1018.0  15732.000000  177.842271  45.000000  1.204364   
1_1024_1061    1024.0  15717.000000  199.655169  48.147100  1.044825   
1_1031_1061    1031.0  15717.000000  193.749199  40.699813  1.144964   
1_1122_1177    1122.0  15805.026316  180.542952  53.852900  0.861529   
1_1138_1177    1138.0  15820.500000  150.066170  46.541353  1.035963   

               Vel_level  Conflict_user  Conflict_id  Angle_level  Distance  \
Conflict_code                                                                 
1_1018_1061          2.0            1.0       1061.0     1.244898  6.062096   
1_1024_1061          2.0            1.0       1061.0     2.000000  7.173504   
1_1031_1061          2.0            1.0       1061.0     1.153846  7.720688   
1_1122_1177          2.0            1.0       1177.0     1.552632  6.449632   
1_1138_1177          

In [76]:
pet = df2.groupby('Conflict_code')['PET'].agg({'pet_mean':np.mean, 'pet_min':np.min, 'pet_max':np.max, 'pet_std': np.std})
dist = df2.groupby('Conflict_code')['Distance'].agg({'dist_mean':np.mean, 'dist_min':np.min, 'dist_max':np.max,'dist_std': np.std})
vel =  df2.groupby('Conflict_code')['Ped_vel'].agg({'vel_mean':np.mean, 'vel_min':np.min, 'vel_max':np.max,'vel_std': np.std})
df3 = (df1.merge(pet, left_on='Conflict', right_on='Conflict_code')
 .merge(dist, left_on='Conflict', right_on='Conflict_code')
 .merge(vel, left_on='Conflict', right_on='Conflict_code'))
df3.to_csv('visual_detection//lda//cluster_result.csv', index=False)
print(df3.loc[df3['cluster']==2,:].max())

C:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
C:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
C:\Software\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


Conflict     2_9939_4848
Topic 0         0.497962
Topic 1       0.00892857
Topic 2         0.507842
Topic 3         0.386483
Topic 4         0.507379
Topic 5         0.167425
Topic 6         0.237216
Topic 7         0.473805
Topic 8       0.00892857
Topic 9       0.00892857
Topic 10         0.99861
Topic 11        0.249584
cluster                2
pet_max          9.99896
pet_min          9.99591
pet_mean         9.99649
pet_std          2.89697
dist_max         32.9467
dist_std         8.06442
dist_mean        27.3297
dist_min         25.8473
vel_max          4.18307
vel_mean         2.84111
vel_min          2.79637
vel_std         0.818316
dtype: object


In [ ]:
conflict_df = pd.read_csv(path_dir + 'filter_conflicts.csv')
g = conflict_df.groupby('Conflict_code')['Word'].apply(list)
for idx, row in g.iterrows():
    print(idx, row)
    

Conflict_code
1_1018_1061    [1_2_1_8, 1_2_1_8, 1_2_1_8, 1_2_1_8, 1_2_1_8, ...
1_1024_1061    [1_2_2_8, 1_2_2_8, 1_2_2_8, 1_2_2_7, 1_2_2_7, ...
1_1031_1061    [1_2_1_6, 1_2_1_6, 1_2_1_6, 1_2_1_6, 1_2_1_7, ...
1_1122_1177    [1_2_2_10, 1_2_2_10, 1_2_1_10, 1_2_1_10, 1_2_1...
1_1138_1177    [1_2_2_9, 1_2_2_9, 1_2_2_9, 1_2_2_9, 1_2_2_9, ...
1_1246_2416    [1_2_2_8, 1_2_2_8, 1_2_2_7, 1_2_2_7, 1_2_2_7, ...
1_1246_2431    [1_2_2_10, 1_2_2_10, 1_2_2_10, 1_2_2_10, 1_2_2...
1_1257_2416    [1_2_1_8, 1_2_1_7, 1_2_1_7, 1_2_1_7, 1_2_1_7, ...
1_1257_2431    [1_2_2_10, 1_2_2_10, 1_2_2_10, 1_2_2_10, 1_2_2...
1_1265_2416    [1_2_2_5, 1_2_2_5, 1_2_2_5, 1_2_2_5, 1_2_2_5, ...
1_1265_2431    [1_2_1_10, 1_2_1_10, 1_2_1_10, 1_2_1_9, 1_2_1_...
1_1273_2408    [1_3_1_10, 1_3_1_10, 1_3_1_10, 1_3_1_10, 1_3_1...
1_1273_2416    [1_2_2_8, 1_2_2_8, 1_2_2_7, 1_2_2_6, 1_2_2_5, ...
1_1273_2431    [1_2_2_10, 1_2_2_10, 1_2_2_10, 1_2_2_10, 1_2_2...
1_1311_2416    [1_2_2_5, 1_2_2_5, 1_2_2_5, 1_2_2_5, 1_2_2_5, ...
1_1311_2431

KeyError: 'Conflict_code'